# Model definition

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Define the transformer model
class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(d_model, input_dim)
        
    def forward(self, x):
        x = x.transpose(0,1)
        x = self.transformer(x)
        x = self.fc(x)
        return x.transpose(0,1)

# Create an instance of the model
input_dim = 1
d_model = 128
nhead = 2
num_layers = 3
model = SimpleTransformer(input_dim, d_model, nhead, num_layers)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



# Reshape the data to be 3-dimensional in the format (batch_size, time_steps, input_dim)
X_train = X_train.reshape(-1, time_steps, input_dim)
X_train = torch.from_numpy(X_train).float()

# Set the model to training mode
model.train()

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    y_pred = model(X_train)

    # Compute the loss
    loss = criterion(y_pred, X_train)

    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
